VERSION-1

In [8]:
import torch
import torch.nn as nn
import torch.optim as optim
import torchvision
import torchvision.transforms as transforms
from torch.utils.data import DataLoader
from torch.cuda.amp import autocast, GradScaler
from torch.optim.lr_scheduler import OneCycleLR
from tqdm import tqdm
import time

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Using device: {device}")


print(f"PyTorch Version: {torch.__version__}")
use_compile = hasattr(torch, 'compile')

Using device: cuda
PyTorch Version: 2.8.0+cu126


In [9]:
config = {

    "image_size": 32,
    "in_channels": 3,
    "num_classes": 10,


    "patch_size": 4,
    "embed_dim": 512,
    "depth": 6,
    "heads": 8,
    "mlp_dim": 1024,
    "dropout": 0.1,


    "learning_rate": 1e-3,
    "weight_decay": 5e-5,
    "batch_size": 256,
    "epochs": 50,
}


config["num_patches"] = (config["image_size"] // config["patch_size"]) ** 2

In [10]:

transform_train = transforms.Compose([
    transforms.RandomCrop(config["image_size"], padding=4),
    transforms.RandomHorizontalFlip(),
    transforms.RandAugment(num_ops=2, magnitude=9),
    transforms.ToTensor(),
    transforms.Normalize((0.4914, 0.4822, 0.4465), (0.2023, 0.1994, 0.2010)),
    transforms.RandomErasing(p=0.25, scale=(0.02, 0.33), ratio=(0.3, 3.3), value=0, inplace=False),
])

transform_test = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize((0.4914, 0.4822, 0.4465), (0.2023, 0.1994, 0.2010)),
])


print("Downloading and setting up datasets...")
trainset = torchvision.datasets.CIFAR10(
    root='./data', train=True, download=True, transform=transform_train)
trainloader = DataLoader(
    trainset, batch_size=config["batch_size"], shuffle=True, num_workers=4, pin_memory=True)

testset = torchvision.datasets.CIFAR10(
    root='./data', train=False, download=True, transform=transform_test)
testloader = DataLoader(
    testset, batch_size=config["batch_size"], shuffle=False, num_workers=4, pin_memory=True)
print("Data setup complete.")

/usr/local/lib/python3.12/dist-packages/torch/utils/data/dataloader.py:627: UserWarning: This DataLoader will create 4 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  warnings.warn(


Data setup complete.


In [11]:
class PatchEmbedding(nn.Module):
    def __init__(self, config):
        super().__init__()
        self.patch_size = config["patch_size"]
        self.embed_dim = config["embed_dim"]
        self.projection = nn.Conv2d(config["in_channels"], self.embed_dim, kernel_size=self.patch_size, stride=self.patch_size)
        self.cls_token = nn.Parameter(torch.randn(1, 1, self.embed_dim))
        self.positional_embedding = nn.Parameter(torch.randn(1, config["num_patches"] + 1, self.embed_dim))

    def forward(self, x):
        x = self.projection(x).flatten(2).transpose(1, 2)
        cls_tokens = self.cls_token.expand(x.shape[0], -1, -1)
        x = torch.cat((cls_tokens, x), dim=1)
        x += self.positional_embedding
        return x

class TransformerEncoder(nn.Module):
    def __init__(self, embed_dim, heads, mlp_dim, dropout):
        super().__init__()
        self.norm1 = nn.LayerNorm(embed_dim)
        self.attention = nn.MultiheadAttention(embed_dim, heads, dropout=dropout, batch_first=True)
        self.norm2 = nn.LayerNorm(embed_dim)
        self.mlp = nn.Sequential(
            nn.Linear(embed_dim, mlp_dim),
            nn.GELU(),
            nn.Dropout(dropout),
            nn.Linear(mlp_dim, embed_dim),
            nn.Dropout(dropout)
        )

    def forward(self, x):
        x = x + self.attention(self.norm1(x), self.norm1(x), self.norm1(x))[0]
        x = x + self.mlp(self.norm2(x))
        return x

class VisionTransformer(nn.Module):
    def __init__(self, config):
        super().__init__()
        self.patch_embedding = PatchEmbedding(config)
        self.encoder_layers = nn.ModuleList([
            TransformerEncoder(config["embed_dim"], config["heads"], config["mlp_dim"], config["dropout"])
            for _ in range(config["depth"])
        ])
        self.norm = nn.LayerNorm(config["embed_dim"])
        self.classifier = nn.Linear(config["embed_dim"], config["num_classes"])

    def forward(self, x):
        x = self.patch_embedding(x)
        for layer in self.encoder_layers:
            x = layer(x)
        cls_token_final = self.norm(x[:, 0])
        return self.classifier(cls_token_final)

In [12]:
print("Initializing model...")
model = VisionTransformer(config).to(device)

if use_compile:
    print("Using torch.compile for model optimization.")
    model = torch.compile(model)

criterion = nn.CrossEntropyLoss()
optimizer = optim.AdamW(model.parameters(), lr=config["learning_rate"], weight_decay=config["weight_decay"])
scheduler = OneCycleLR(optimizer, max_lr=config["learning_rate"], steps_per_epoch=len(trainloader), epochs=config["epochs"])
scaler = GradScaler()

print("Model and training components initialized.")

Initializing model...
Using torch.compile for model optimization.
Model and training components initialized.


/tmp/ipython-input-3548584274.py:11: FutureWarning: `torch.cuda.amp.GradScaler(args...)` is deprecated. Please use `torch.amp.GradScaler('cuda', args...)` instead.
  scaler = GradScaler()


In [15]:
def train(epoch):
    model.train()
    train_loss = 0
    correct = 0
    total = 0
    progress_bar = tqdm(trainloader, desc=f"Epoch {epoch+1}/{config['epochs']} [Train]")

    for batch_idx, (inputs, targets) in enumerate(progress_bar):
        inputs, targets = inputs.to(device), targets.to(device)
        optimizer.zero_grad()


        with torch.amp.autocast(device_type=device.type, dtype=torch.float16):
            outputs = model(inputs)
            loss = criterion(outputs, targets)

        scaler.scale(loss).backward()
        scaler.step(optimizer)
        scaler.update()
        scheduler.step()

        train_loss += loss.item()
        _, predicted = outputs.max(1)
        total += targets.size(0)
        correct += predicted.eq(targets).sum().item()

        progress_bar.set_postfix(
            loss=f'{train_loss/(batch_idx+1):.3f}',
            acc=f'{100.*correct/total:.2f}%',
            lr=f'{scheduler.get_last_lr()[0]:.1e}'
        )

def test(epoch):
    global best_acc
    model.eval()
    test_loss = 0
    correct = 0
    total = 0
    progress_bar = tqdm(testloader, desc=f"Epoch {epoch+1}/{config['epochs']} [Test]")
    with torch.no_grad():
        for batch_idx, (inputs, targets) in enumerate(progress_bar):
            inputs, targets = inputs.to(device), targets.to(device)
            outputs = model(inputs)
            loss = criterion(outputs, targets)

            test_loss += loss.item()
            _, predicted = outputs.max(1)
            total += targets.size(0)
            correct += predicted.eq(targets).sum().item()

            progress_bar.set_postfix(
                loss=f'{test_loss/(batch_idx+1):.3f}',
                acc=f'{100.*correct/total:.2f}%'
            )

    acc = 100. * correct / total
    if acc > best_acc:
        print(f'New best accuracy: {acc:.2f}%. Saving model...')
        best_acc = acc


    return acc

In [17]:


print("Initializing model for training...")
model = VisionTransformer(config).to(device)

if use_compile:
    print("Using torch.compile for model optimization.")
    model = torch.compile(model)

criterion = nn.CrossEntropyLoss()
optimizer = optim.AdamW(model.parameters(), lr=config["learning_rate"], weight_decay=config["weight_decay"])
scheduler = OneCycleLR(optimizer, max_lr=config["learning_rate"], steps_per_epoch=len(trainloader), epochs=config["epochs"])

scaler = GradScaler()
print("Model and training components initialized.")


best_acc = 0.0
start_time = time.time()
print("\nStarting training...")

for epoch in range(config['epochs']):
    train(epoch)
    current_acc = test(epoch)

end_time = time.time()
total_time = end_time - start_time

print("\n--- Training Finished ---")
print(f"Total Training Time: {total_time/60:.2f} minutes")
print(f"Best Test Accuracy: {best_acc:.2f}%")

/tmp/ipython-input-827708161.py:16: FutureWarning: `torch.cuda.amp.GradScaler(args...)` is deprecated. Please use `torch.amp.GradScaler('cuda', args...)` instead.
  scaler = GradScaler()


Initializing model for training...
Using torch.compile for model optimization.
Model and training components initialized.

Starting training...


Epoch 1/50 [Test]: 100%|██████████| 40/40 [00:06<00:00,  6.18it/s, acc=32.38%, loss=1.839]


New best accuracy: 32.38%. Saving model...


Epoch 2/50 [Test]: 100%|██████████| 40/40 [00:06<00:00,  6.17it/s, acc=46.22%, loss=1.522]


...

New best accuracy: 83.39%. Saving model...


Epoch 49/50 [Test]: 100%|██████████| 40/40 [00:06<00:00,  6.45it/s, acc=83.45%, loss=0.474]


New best accuracy: 83.45%. Saving model...


Epoch 50/50 [Test]: 100%|██████████| 40/40 [00:06<00:00,  6.43it/s, acc=83.47%, loss=0.474]

New best accuracy: 83.47%. Saving model...

--- Training Finished ---
Total Training Time: 42.66 minutes
Best Test Accuracy: 83.47%


VERSION-2

In [18]:
import torch
import torch.nn as nn
import torch.optim as optim
import torchvision
import torchvision.transforms as transforms
from torchvision.ops import StochasticDepth
from torch.utils.data import DataLoader
from torch.cuda.amp import autocast, GradScaler
from torch.optim.lr_scheduler import OneCycleLR
from tqdm import tqdm
import time


device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Using device: {device}")
use_compile = hasattr(torch, 'compile')
print(f"PyTorch Version: {torch.__version__}, using torch.compile: {use_compile}")

Using device: cuda
PyTorch Version: 2.8.0+cu126, using torch.compile: True


In [19]:
config = {

    "image_size": 32,
    "in_channels": 3,
    "num_classes": 10,


    "patch_size": 4,
    "embed_dim": 512,
    "depth": 6,
    "heads": 8,
    "mlp_dim": 1024,
    "dropout": 0.1,
    "stochastic_depth_prob": 0.1,


    "learning_rate": 1e-3,
    "weight_decay": 5e-5,
    "batch_size": 256,
    "epochs": 100,
    "label_smoothing": 0.1,
}


config["num_patches"] = (config["image_size"] // config["patch_size"]) ** 2

In [20]:

transform_train = transforms.Compose([
    transforms.RandomCrop(config["image_size"], padding=4),
    transforms.RandomHorizontalFlip(),
    transforms.RandAugment(num_ops=2, magnitude=9),
    transforms.ToTensor(),
    transforms.Normalize((0.4914, 0.4822, 0.4465), (0.2023, 0.1994, 0.2010)),
    transforms.RandomErasing(p=0.25),
])

transform_test = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize((0.4914, 0.4822, 0.4465), (0.2023, 0.1994, 0.2010)),
])


print("Downloading and setting up datasets...")
trainset = torchvision.datasets.CIFAR10(
    root='./data', train=True, download=True, transform=transform_train)
trainloader = DataLoader(
    trainset, batch_size=config["batch_size"], shuffle=True, num_workers=4, pin_memory=True)

testset = torchvision.datasets.CIFAR10(
    root='./data', train=False, download=True, transform=transform_test)
testloader = DataLoader(
    testset, batch_size=config["batch_size"], shuffle=False, num_workers=4, pin_memory=True)
print("Data setup complete.")

Data setup complete.


In [21]:
class PatchEmbedding(nn.Module):

    def __init__(self, config):
        super().__init__()
        self.patch_size = config["patch_size"]
        self.embed_dim = config["embed_dim"]
        self.projection = nn.Conv2d(config["in_channels"], self.embed_dim, kernel_size=self.patch_size, stride=self.patch_size)
        self.cls_token = nn.Parameter(torch.randn(1, 1, self.embed_dim))
        self.positional_embedding = nn.Parameter(torch.randn(1, config["num_patches"] + 1, self.embed_dim))

    def forward(self, x):
        x = self.projection(x).flatten(2).transpose(1, 2)
        cls_tokens = self.cls_token.expand(x.shape[0], -1, -1)
        x = torch.cat((cls_tokens, x), dim=1)
        x += self.positional_embedding
        return x

class TransformerEncoder(nn.Module):

    def __init__(self, embed_dim, heads, mlp_dim, dropout, stochastic_depth_prob):
        super().__init__()
        self.norm1 = nn.LayerNorm(embed_dim)
        self.attention = nn.MultiheadAttention(embed_dim, heads, dropout=dropout, batch_first=True)
        self.norm2 = nn.LayerNorm(embed_dim)
        self.mlp = nn.Sequential(
            nn.Linear(embed_dim, mlp_dim),
            nn.GELU(),
            nn.Dropout(dropout),
            nn.Linear(mlp_dim, embed_dim),
            nn.Dropout(dropout)
        )
        self.stochastic_depth = StochasticDepth(stochastic_depth_prob, "row")

    def forward(self, x):
        attn_output = self.attention(self.norm1(x), self.norm1(x), self.norm1(x))[0]
        x = x + self.stochastic_depth(attn_output)
        mlp_output = self.mlp(self.norm2(x))
        x = x + self.stochastic_depth(mlp_output)
        return x

class VisionTransformer(nn.Module):
    def __init__(self, config):
        super().__init__()
        self.patch_embedding = PatchEmbedding(config)


        self.encoder_layers = nn.ModuleList([
            TransformerEncoder(
                config["embed_dim"],
                config["heads"],
                config["mlp_dim"],
                config["dropout"],
                config["stochastic_depth_prob"]
            )
            for _ in range(config["depth"])
        ])

        self.norm = nn.LayerNorm(config["embed_dim"])
        self.classifier = nn.Linear(config["embed_dim"], config["num_classes"])

    def forward(self, x):
        x = self.patch_embedding(x)
        for layer in self.encoder_layers:
            x = layer(x)
        cls_token_final = self.norm(x[:, 0])
        return self.classifier(cls_token_final)

In [22]:


def train(epoch):
    model.train()
    train_loss = 0
    correct = 0
    total = 0
    progress_bar = tqdm(trainloader, desc=f"Epoch {epoch+1}/{config['epochs']} [Train]")

    for batch_idx, (inputs, targets) in enumerate(progress_bar):
        inputs, targets = inputs.to(device), targets.to(device)
        optimizer.zero_grad()

        with torch.amp.autocast(device_type=device.type, dtype=torch.float16):
            outputs = model(inputs)
            loss = criterion(outputs, targets)

        scaler.scale(loss).backward()
        scaler.step(optimizer)
        scaler.update()
        scheduler.step()

        train_loss += loss.item()
        _, predicted = outputs.max(1)
        total += targets.size(0)
        correct += predicted.eq(targets).sum().item()

        progress_bar.set_postfix(
            loss=f'{train_loss/(batch_idx+1):.3f}',
            acc=f'{100.*correct/total:.2f}%',
            lr=f'{scheduler.get_last_lr()[0]:.1e}'
        )

def test(epoch):
    global best_acc
    model.eval()
    test_loss = 0
    correct = 0
    total = 0
    progress_bar = tqdm(testloader, desc=f"Epoch {epoch+1}/{config['epochs']} [Test]")
    with torch.no_grad():
        for batch_idx, (inputs, targets) in enumerate(progress_bar):
            inputs, targets = inputs.to(device), targets.to(device)
            outputs = model(inputs)
            loss = criterion(outputs, targets)

            test_loss += loss.item()
            _, predicted = outputs.max(1)
            total += targets.size(0)
            correct += predicted.eq(targets).sum().item()

            progress_bar.set_postfix(
                loss=f'{test_loss/(batch_idx+1):.3f}',
                acc=f'{100.*correct/total:.2f}%'
            )

    acc = 100. * correct / total
    if acc > best_acc:
        print(f'New best accuracy: {acc:.2f}%.')
        best_acc = acc

    return acc

In [23]:
print("Initializing model for training...")
model = VisionTransformer(config).to(device)

if use_compile:
    print("Using torch.compile for model optimization.")
    model = torch.compile(model)

criterion = nn.CrossEntropyLoss(label_smoothing=config["label_smoothing"])

optimizer = optim.AdamW(model.parameters(), lr=config["learning_rate"], weight_decay=config["weight_decay"])
scheduler = OneCycleLR(optimizer, max_lr=config["learning_rate"], steps_per_epoch=len(trainloader), epochs=config["epochs"])
scaler = torch.amp.GradScaler(device.type)

print("Model and training components initialized.")


best_acc = 0.0
start_time = time.time()
print("\nStarting training...")

for epoch in range(config['epochs']):
    train(epoch)
    current_acc = test(epoch)

end_time = time.time()
total_time = end_time - start_time

print("\n--- Training Finished ---")
print(f"Total Training Time: {total_time/60:.2f} minutes")
print(f"Best Test Accuracy: {best_acc:.2f}%")

Initializing model for training...
Using torch.compile for model optimization.
Model and training components initialized.

Starting training...


Epoch 1/100 [Test]:   0%|          | 0/40 [00:00<?, ?it/s]/usr/local/lib/python3.12/dist-packages/torch/utils/data/dataloader.py:627: UserWarning: This DataLoader will create 4 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  warnings.warn(
Epoch 1/100 [Test]: 100%|██████████| 40/40 [00:12<00:00,  3.30it/s, acc=33.34%, loss=1.897]


New best accuracy: 33.34%.


Epoch 2/100 [Train]:   0%|          | 0/196 [00:00<?, ?it/s]/usr/local/lib/python3.12/dist-packages/torch/utils/data/dataloader.py:627: UserWarning: This DataLoader will create 4 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  warnings.warn(
Epoch 2/100 [Test]: 100%|██████████| 40/40 [00:06<00:00,  6.44it/s, acc=42.55%, loss=1.756]


New best accuracy: 42.55%.


...

New best accuracy: 88.16%.


Epoch 89/100 [Test]: 100%|██████████| 40/40 [00:06<00:00,  6.30it/s, acc=88.37%, loss=0.770]


New best accuracy: 88.37%.


Epoch 100/100 [Test]: 100%|██████████| 40/40 [00:06<00:00,  6.32it/s, acc=88.23%, loss=0.765]


--- Training Finished ---
Total Training Time: 86.02 minutes
Best Test Accuracy: 88.37%
